## **Langchain을 표현하는 언어, LCEL**

**[LCEL로 기본 체인 구성하기]**

In [ ]:
import os
os.environ["OPENAI_API_KEY"] = "YOUR_API_KEY"

In [ ]:
#필수 라이브러리 설치
!pip install --upgrade --quiet langchain openai langchain-core langchain-openai

In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

#프롬프트 템플릿 설정
prompt = ChatPromptTemplate.from_template("tell me a short joke about {topic}")

#LLM 호출
model = ChatOpenAI(model="gpt-4o-mini")

#출력 파서 설정
output_parser = StrOutputParser()

#LCEL로 프롬프트템플릿-LLM-출력 파서 연결하기
chain = prompt | model | output_parser

#invoke함수로 chain 실행하기
chain.invoke({"topic": "ice cream"})

**[Streaming 기능 추가를 더욱 쉽게, stream()]**

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

#Chain 선언
model = ChatOpenAI(model="gpt-4o-mini")
prompt = ChatPromptTemplate.from_template("tell me a joke about {topic}")
chain = prompt | model

#Chain의 stream()함수를 통해 스트리밍 기능 추가
for s in chain.stream({"topic": "bears"}):
    print(s.content, end="", flush=True)

**[한꺼번에 여러 개 API 요청하고 답변 받기, batch()]**

- 5개 문장 번역 batch 수행

In [ ]:
%%time
model = ChatOpenAI(model="gpt-4o-mini")
prompt = ChatPromptTemplate.from_template("다음 한글 문장을 프랑스어로 번역해줘 {sentence}")
chain = prompt | model

chain.batch([
    {"sentence": "그녀는 매일 아침 책을 읽습니다."},
    {"sentence": "오늘 날씨가 참 좋네요."},
    {"sentence": "저녁에 친구들과 영화를 볼 거예요."},
    {"sentence": "그 학생은 매우 성실하게 공부합니다."},
    {"sentence": "커피 한 잔이 지금 딱 필요해요."}

])

- 1개 문장 번역 invoke 수행

In [ ]:
%%time
prompt = ChatPromptTemplate.from_template("다음 한글 문장을 프랑스어로 번역해줘 {sentence}")
chain = prompt | model

chain.invoke({"sentence": "그녀는 매일 아침 책을 읽습니다."})

## **RunnablePassthrough, RunnableLambda, RunnableParallel**

**[RunnablePassthrough]**

**RunnablePassthrough는 가장 단순한 Runnable 객체로, 들어온 입력을 그대로 전달합니다.**

In [ ]:
pip install -q langchain langchain-core langchain-openai

In [ ]:
import os
os.environ["OPENAI_API_KEY"] = "YOUR_API_KEY"

In [ ]:
from langchain_core.runnables import RunnablePassthrough

RunnablePassthrough().invoke("안녕하세요")

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

prompt = ChatPromptTemplate.from_template("다음 한글 문장을 프랑스어로 번역해줘 {sentence} \n French sentence: (print from here)")
model = ChatOpenAI(model="gpt-4o-mini")
output_parser = StrOutputParser()

runnable_chain = {"sentence": RunnablePassthrough()} | prompt | model | output_parser

runnable_chain.invoke({"sentence": "그녀는 매일 아침 책을 읽습니다."})

RunnablePassthrough는 assin 함수를 통해 새로운 변수에 계산된 값을 입력할 수 있습니다.

In [ ]:
(RunnablePassthrough.assign(mult=lambda x: x["num"]*3)).invoke({"num":3})

In [ ]:
from langchain_core.runnables import RunnableParallel, RunnablePassthrough

runnable = RunnableParallel(
    extra=RunnablePassthrough.assign(mult=lambda x: x["num"] * 3),
    modified=lambda x: x["num"] + 1,
)

runnable.invoke({"num": 1})

**[RunnableLambda]**

**RunnableLambda는 임의의 함수를 Chain에 결합할 수 있게 Runnable 객체로 변환합니다..**

In [ ]:
def add_smile(x):
    return x + ":)"

In [ ]:
from langchain_core.runnables import RunnableLambda

add_smile = RunnableLambda(add_smile)

In [ ]:
from langchain.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain.schema.output_parser import StrOutputParser

prompt_str = "{topic}의 역사에 대해 세문장으로 설명해주세요."
prompt = ChatPromptTemplate.from_template(prompt_str)

model = ChatOpenAI(model_name = 'gpt-4o-mini')

output_parser = StrOutputParser()

chain = prompt | model | output_parser

In [ ]:
from langchain_core.runnables import RunnableLambda

def add_thank(x):
    return x + " 들어주셔서 감사합니다 :)"

add_thank = RunnableLambda(add_thank)

In [ ]:
chain = prompt | model | output_parser | add_thank
chain.invoke("반도체")

In [ ]:
pip install -q grandalf

In [ ]:
chain.get_graph().print_ascii()

**[RunnableParallel]**

**RunnableParallel은 여러 요소가 병렬 처리되도록 처리합니다.**

In [ ]:
from langchain_core.runnables import RunnableParallel, RunnablePassthrough

runnable = RunnableParallel(
    passed=RunnablePassthrough(),
    modified=lambda x: x["num"] + 1,
)

runnable.invoke({"num": 1})

In [ ]:
runnable = RunnableParallel(
    passed=RunnablePassthrough(),
    modified=add_thank,
)

In [ ]:
runnable.invoke("안녕하세요")

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableParallel
from langchain_openai import ChatOpenAI
from langchain.schema.output_parser import StrOutputParser

model = ChatOpenAI(model = 'gpt-4o-mini', max_tokens = 128, temperature = 0)

history_prompt = ChatPromptTemplate.from_template("{topic}가 무엇의 약자인지 알려주세요.")
celeb_prompt = ChatPromptTemplate.from_template("{topic} 분야의 유명인사 3명의 이름만 알려주세요.")

output_parser = StrOutputParser()

history_chain = history_prompt | model | output_parser
celeb_chain = celeb_prompt | model | output_parser

map_chain = RunnableParallel(history=history_chain, celeb=celeb_chain)

result = map_chain.invoke({"topic": "AI"})

In [ ]:
result

## **LCEL로 RAG 구축하기**

**[기본 RAG 구축]**

In [ ]:
pip install -q langchain_chroma langchain_community langchainhub

In [ ]:
from langchain_openai import ChatOpenAI
import bs4
from langchain import hub
from langchain_chroma import Chroma
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

model = ChatOpenAI(model="gpt-4o-mini")
# Load, chunk and index the contents of the blog.
loader = WebBaseLoader(
    web_paths=("https://n.news.naver.com/mnews/article/366/0001007619",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("newsct_article _article_body")
        )
    ),
)
docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=300, chunk_overlap=50)
splits = text_splitter.split_documents(docs)
vectorstore = Chroma.from_documents(documents=splits, embedding=OpenAIEmbeddings())

# Retrieve and generate using the relevant snippets of the blog.
retriever = vectorstore.as_retriever()
prompt = hub.pull("rlm/rag-prompt")


def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

rag_chain.invoke("아까 내가 했던 질문이 뭐야?")

'죄송하지만, 아까 질문하신 내용을 기억할 수 없습니다.'

## **메모리를 추가한 RAG 구축하기**

In [ ]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import MessagesPlaceholder
from langchain_core.messages import HumanMessage

loader = WebBaseLoader(
    web_paths=("https://n.news.naver.com/mnews/article/366/0001007619",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("newsct_article _article_body")
        )
    ),
)
docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)
vectorstore = Chroma.from_documents(documents=splits, embedding=OpenAIEmbeddings())
retriever = vectorstore.as_retriever()

In [ ]:
contextualize_q_system_prompt = (
    "Given a chat history and the latest user question "
    "which might reference context in the chat history, "
    "formulate a standalone question which can be understood "
    "without the chat history. Do NOT answer the question, "
    "just reformulate it if needed and otherwise return it as is."
)

contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)

In [ ]:
history_aware_retriever = create_history_aware_retriever(
    model, retriever, contextualize_q_prompt
)

In [ ]:
system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)
qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)
question_answer_chain = create_stuff_documents_chain(model, qa_prompt)

rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)

In [ ]:

chat_history = []

question = "오픈AI 근황은 어때?"
ai_msg_1 = rag_chain.invoke({"input": question, "chat_history": chat_history})
chat_history.extend([HumanMessage(content=question), ai_msg_1["answer"]])
ai_msg_1

{'input': '오픈AI 근황은 어때?',
 'chat_history': [HumanMessage(content='오픈AI 근황은 어때?'),
  "오픈AI는 최근 자체 검색 엔진 '서치GPT'를 공개하고 일부 이용자 대상으로 테스트를 시작했습니다. 이로 인해 구글의 주가가 약 3% 하락하는 등 검색 시장에서의 도전에 나섰습니다. 오픈AI는 서치GPT를 챗GPT와 통합할 계획을 세우고 있습니다."],
 'context': [Document(metadata={'source': 'https://n.news.naver.com/mnews/article/366/0001007619'}, page_content='생성형 인공지능(AI) 챗GPT 개발사 오픈AI가 검색 왕국 구글에 도전장을 내밀었다.'),
  Document(metadata={'source': 'https://n.news.naver.com/mnews/article/366/0001007619'}, page_content='생성형 인공지능(AI) 챗GPT 개발사 오픈AI가 검색 왕국 구글에 도전장을 내밀었다.'),
  Document(metadata={'source': 'https://n.news.naver.com/mnews/article/366/0001007619'}, page_content='생성형 인공지능(AI) 챗GPT 개발사 오픈AI가 검색 왕국 구글에 도전장을 내밀었다.\n\n\n\n오픈AI 로고 / AP=연합뉴스        25일(현지 시각) 오픈AI는 자체 검색 엔진 ‘서치GPT’(SearchGPT)를 공개하며 일부 이용자를 대상으로 테스트에 들어갔다. 오픈AI의 검색 시장 도전으로 글로벌 최대 검색 업체인 구글의 주가는 전날 대비 3% 가량 하락했다.이날 오픈AI가 공개한 데모 속 서치GPT는 마치 구글 홈페이지처럼 간단한 로고와 함께 질문을 입력할 수 있는 창이 중앙에 떠있는 모습이다. 검색어를 입력하면 관련 정보와 참고된 웹사이트들이 표시되는 모습은 구글과 큰 차이가 없지만, 서치GPT

In [ ]:
chat_history

[HumanMessage(content='오픈AI 근황은 어때?'),
 "오픈AI는 최근 자체 검색 엔진 '서치GPT'를 공개하고 일부 이용자 대상으로 테스트를 시작했습니다. 이로 인해 구글의 주가가 약 3% 하락하는 등 검색 시장에서의 도전에 나섰습니다. 오픈AI는 서치GPT를 챗GPT와 통합할 계획을 세우고 있습니다."]

In [ ]:
question = "다시 한번 설명해줄래?"
ai_msg_2 = rag_chain.invoke({"input": question, "chat_history": chat_history})
chat_history.extend([HumanMessage(content=question), ai_msg_2["answer"]])
ai_msg_2

{'input': '다시 한번 설명해줄래?',
 'chat_history': [HumanMessage(content='오픈AI 근황은 어때?'),
  "오픈AI는 최근 자체 검색 엔진 '서치GPT'를 공개하고 일부 이용자 대상으로 테스트를 시작했습니다. 이로 인해 구글의 주가가 약 3% 하락하는 등 검색 시장에서의 도전에 나섰습니다. 오픈AI는 서치GPT를 챗GPT와 통합할 계획을 세우고 있습니다.",
  HumanMessage(content='다시 한번 설명해줄래?'),
  "오픈AI는 최근 자체 검색 엔진 '서치GPT'를 공개하고 일부 이용자를 대상으로 테스트를 시작했습니다. 이로 인해 구글의 주가가 약 3% 하락하며 검색 시장에서 도전에 나섰습니다. 오픈AI는 서치GPT를 자사의 챗GPT와 통합할 계획입니다."],
 'context': [Document(metadata={'source': 'https://n.news.naver.com/mnews/article/366/0001007619'}, page_content='생성형 인공지능(AI) 챗GPT 개발사 오픈AI가 검색 왕국 구글에 도전장을 내밀었다.'),
  Document(metadata={'source': 'https://n.news.naver.com/mnews/article/366/0001007619'}, page_content='생성형 인공지능(AI) 챗GPT 개발사 오픈AI가 검색 왕국 구글에 도전장을 내밀었다.'),
  Document(metadata={'source': 'https://n.news.naver.com/mnews/article/366/0001007619'}, page_content='생성형 인공지능(AI) 챗GPT 개발사 오픈AI가 검색 왕국 구글에 도전장을 내밀었다.\n\n\n\n오픈AI 로고 / AP=연합뉴스        25일(현지 시각) 오픈AI는 자체 검색 엔진 ‘서치GPT’(SearchGPT)를 공개하며 일부 이용자를 대상으로 테스트에 들어갔다. 오픈AI의 검색 시장 

In [ ]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store = {}

def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]


conversational_rag_chain = RunnableWithMessageHistory(
    rag_chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
    output_messages_key="answer",
)

In [ ]:
conversational_rag_chain.invoke(
    {"input": "오픈AI의 근황이 어때?"},
    config={
        "configurable": {"session_id": "abc123"}
    },  # constructs a key "abc123" in `store`.
)["answer"]

"오픈AI는 최근 자체 검색 엔진 '서치GPT'를 공개하고 일부 이용자를 대상으로 테스트를 시작했습니다. 이로 인해 구글의 주가는 약 3% 하락했으며, 오픈AI는 서치GPT를 챗GPT와 통합할 계획을 가지고 있습니다. 샘 올트먼 CEO는 더 나은 검색을 만들 여지가 있다고 언급했습니다."

In [ ]:
conversational_rag_chain.invoke(
    {"input": "앞으로 어떻게 될까?"},
    config={"configurable": {"session_id": "abc123"}},
)["answer"]

'오픈AI의 서치GPT와 같은 AI 기반 검색 기술은 사용자 경험을 개선하고 정보 검색 방식을 혁신할 가능성이 큽니다. AI 헬스케어 시장과 같은 다른 분야에서도 AI 기술의 적용이 증가할 것으로 예상되며, 이는 다양한 산업에 긍정적인 영향을 미칠 수 있습니다. 그러나 기술 발전에 따른 윤리적, 사회적 문제도 함께 고려해야 할 것입니다.'

In [ ]:
from langchain_core.messages import AIMessage

for message in store["abc123"].messages:
    if isinstance(message, AIMessage):
        prefix = "AI"
    else:
        prefix = "User"

    print(f"{prefix}: {message.content}\n")

User: 오픈AI의 근황이 어때?

AI: 오픈AI는 최근 자체 검색 엔진 '서치GPT'를 공개하고 일부 이용자를 대상으로 테스트를 시작했습니다. 이로 인해 구글의 주가는 약 3% 하락했으며, 오픈AI는 서치GPT를 챗GPT와 통합할 계획을 가지고 있습니다. 샘 올트먼 CEO는 더 나은 검색을 만들 여지가 있다고 언급했습니다.

User: 앞으로 어떻게 될까?

AI: 오픈AI의 서치GPT와 같은 AI 기반 검색 기술은 사용자 경험을 개선하고 정보 검색 방식을 혁신할 가능성이 큽니다. AI 헬스케어 시장과 같은 다른 분야에서도 AI 기술의 적용이 증가할 것으로 예상되며, 이는 다양한 산업에 긍정적인 영향을 미칠 수 있습니다. 그러나 기술 발전에 따른 윤리적, 사회적 문제도 함께 고려해야 할 것입니다.

